In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import pandas as pd
import numpy as np
import os

from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

import gc
import pickle as pickle


from sklearn.model_selection import KFold
from itertools import product
import tensorflow as tf

In [ ]:
#tail -n +2 train.csv | split -l 150000


In [6]:
train = pd.read_csv('train/xaa', float_precision='round_trip', header=None)
train.columns = ['acoustic_data','time_to_failure']

In [7]:
with pd.option_context('display.precision', 15):
    print(train.head())
    print(train.tail())

   acoustic_data  time_to_failure
0             12     1.4690999832
1              6     1.4690999821
2              8     1.4690999810
3              5     1.4690999799
4              8     1.4690999788
        acoustic_data  time_to_failure
149995              1     1.4307971903
149996              6     1.4307971892
149997              6     1.4307971881
149998              2     1.4307971870
149999              0     1.4307971859


In [8]:
all_features = ['acoustic_data', 'rolling_100']

In [9]:
features = all_features

In [10]:
train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 2 columns):
acoustic_data      150000 non-null int64
time_to_failure    150000 non-null float64
dtypes: float64(1), int64(1)
memory usage: 2.3 MB


In [12]:
from os import listdir
from os.path import isfile, join

TRAIN_SPLITS='train'
splits = [f for f in listdir(TRAIN_SPLITS) if isfile(join(TRAIN_SPLITS, f))]

In [13]:
splits

['xzbvh',
 'xvv',
 'xzarc',
 'xzfad',
 'xum',
 'xhv',
 'xzdcy',
 'xiz',
 'xxn',
 'xzdlg',
 'xzanz',
 'xzeln',
 'xzagt',
 'xzdle',
 'xng',
 'xzbhs',
 'xzbko',
 'xzcsr',
 'xzckz',
 'xzdme',
 'xjd',
 'xtz',
 'xzaju',
 'xzavo',
 'xzbkq',
 'xyv',
 'xzbpt',
 'xzckn',
 'xzeog',
 'xzapz',
 'xzbdg',
 'xhx',
 'xzcrq',
 'xzbok',
 'xzejd',
 'xec',
 'xzcgi',
 'xhm',
 'xzbmd',
 'xcg',
 'xzeyt',
 'xzcdz',
 'xzbpr',
 'xln',
 'xzbzl',
 'xzcbv',
 'xzeia',
 'xzdoa',
 'xzffk',
 'xzbcd',
 'xzaae',
 'xzcik',
 'xzehd',
 'xzbck',
 'xzcnw',
 'xzdcn',
 'xzcvx',
 'xzcoy',
 'xzfff',
 'xzagf',
 'xjv',
 'xzdln',
 'xzciv',
 'xzahe',
 'xzbia',
 'xzclr',
 'xzdgh',
 'xzcir',
 'xzecg',
 'xzdfm',
 'xth',
 'xzbmj',
 'xzdwi',
 'xzera',
 'xzedf',
 'xhu',
 'xzasm',
 'xzehw',
 'xzboz',
 'xzffm',
 'xzbqb',
 'xzaua',
 'xzbhl',
 'xzdze',
 'xzabi',
 'xzcmm',
 'xzdha',
 'xzahq',
 'xzffq',
 'xzaoy',
 'xzdkz',
 'xzble',
 'xzdjh',
 'xjp',
 'xah',
 'xzevq',
 'xzcsc',
 'xzcmu',
 'xzbgf',
 'xzehx',
 'xhg',
 'xei',
 'xzbay',
 'xzdhm',
 '

In [14]:
splits_chunk = splits[0:101]

In [16]:
training = []
ys = []

from sklearn.preprocessing import StandardScaler,MinMaxScaler 

for chunk in splits_chunk:
    path = 'train/%s' % (chunk)
    chunk_df = pd.read_csv(path, float_precision='round_trip', header=None)
    chunk_df.columns = ['acoustic_data','time_to_failure']
    chunk_df['rolling_100'] = chunk_df['acoustic_data'].rolling(window=100, min_periods=1).mean()#.apply(np.log)
    chunk_df[all_features] = StandardScaler().fit_transform(chunk_df[all_features])
    training.append(chunk_df[all_features].values)
    ys.append(chunk_df['time_to_failure'].values[-1])

training = np.array(training)
ys = np.array(ys)

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/base.py:464: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/shemery/.local/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversi

In [17]:
ys.shape

(101,)

In [18]:
training.shape

(101, 150000, 2)

In [ ]:
#number of batches must cover a sequence length i.e. 100 * 1500 = 15000, batch size = 1500 

In [31]:
split_size = 500
new_training = []
for sequence in training:
    #split each sequence in split_size
    splits = np.array_split(sequence, split_size)
    new_training.append(splits)
    
new_training = np.array(new_training)

In [56]:
training.shape

(101, 150000, 2)

In [35]:
new_training.shape

(101, 500, 300, 2)

In [88]:
np.swapaxes(new_training,0,1).shape

(500, 101, 300, 2)

In [100]:
a = np.array([[[[1],[2],[3],[4]],[[5],[6],[7],[8]],[[8],[9],[10],[11]]], [[[10],[20],[30],[40]],[[50],[60],[70],[80]],[[80],[90],[100],[110]]],\
              [[[1000],[2000],[3000],[4000]],[[5000],[6000],[7000],[8000]],[[8000],[9000],[10000],[11000]]] ])
print(a.shape)
a

(3, 3, 4, 1)


array([[[[    1],
         [    2],
         [    3],
         [    4]],

        [[    5],
         [    6],
         [    7],
         [    8]],

        [[    8],
         [    9],
         [   10],
         [   11]]],


       [[[   10],
         [   20],
         [   30],
         [   40]],

        [[   50],
         [   60],
         [   70],
         [   80]],

        [[   80],
         [   90],
         [  100],
         [  110]]],


       [[[ 1000],
         [ 2000],
         [ 3000],
         [ 4000]],

        [[ 5000],
         [ 6000],
         [ 7000],
         [ 8000]],

        [[ 8000],
         [ 9000],
         [10000],
         [11000]]]])

In [101]:
np.swapaxes(a,0,1).shape

(3, 3, 4, 1)

In [105]:
np.swapaxes(a,0,1).reshape(9,4,1)

array([[[    1],
        [    2],
        [    3],
        [    4]],

       [[   10],
        [   20],
        [   30],
        [   40]],

       [[ 1000],
        [ 2000],
        [ 3000],
        [ 4000]],

       [[    5],
        [    6],
        [    7],
        [    8]],

       [[   50],
        [   60],
        [   70],
        [   80]],

       [[ 5000],
        [ 6000],
        [ 7000],
        [ 8000]],

       [[    8],
        [    9],
        [   10],
        [   11]],

       [[   80],
        [   90],
        [  100],
        [  110]],

       [[ 8000],
        [ 9000],
        [10000],
        [11000]]])

In [44]:
new_training.reshape(-1,300,2)[0]

array([[-1.24873569e+00, -1.55934040e+01],
       [-7.57097039e-02, -8.26889608e+00],
       [-4.66718366e-01, -7.45506186e+00],
       [-4.66718366e-01, -7.04814475e+00],
       [-2.71214035e-01, -6.31569396e+00],
       [-1.05323136e+00, -7.45506186e+00],
       [-7.57097039e-02, -6.52496561e+00],
       [ 1.29282061e+00, -3.69107861e+00],
       [ 5.10803289e-01, -2.57205657e+00],
       [ 1.09731628e+00, -9.44388134e-01],
       [ 7.06307620e-01, -5.65689896e-02],
       [ 1.19794627e-01,  7.29046357e-02],
       [-7.57097039e-02, -5.34865433e-03],
       [-4.66718366e-01, -4.21208995e-01],
       [-8.57727028e-01, -1.10715498e+00],
       [-4.66718366e-01, -1.40216988e+00],
       [-6.62222697e-01, -1.80609495e+00],
       [-1.44424002e+00, -2.70769560e+00],
       [-1.44424002e+00, -3.51439092e+00],
       [-1.05323136e+00, -3.99626644e+00],
       [-1.05323136e+00, -4.43224906e+00],
       [ 5.10803289e-01, -3.94077775e+00],
       [ 1.29282061e+00, -3.06743391e+00],
       [ 1.

In [39]:
training[0].reshape(-1, split_size, 2).T.shape

(2, 500, 300)

In [25]:
np.array(np.array_split(training[0], 1000)).shape

(1000, 150, 2)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data, y_train, y_val = train_test_split(training, ys, test_size=0.1, random_state=42)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Flatten,GRU,CuDNNGRU,CuDNNLSTM
from keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization


dropout=0.2

my_model = Sequential()

my_model.add(LSTM(use_bias = True,unit_forget_bias=True,units = 3,\
                  dropout=dropout,recurrent_dropout=dropout,input_shape = (training.shape[1],len(features))))
my_model.add(Dense(1))

my_model.compile(loss = 'mae',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()




callbacks = [
    EarlyStopping(monitor='val_loss', patience=0, verbose=0)
]

history = my_model.fit(train_data, y_train, batch_size=3, epochs=100,
                      validation_data=(val_data,y_val), callbacks=callbacks
                      )


import matplotlib.pyplot as plt


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

import math
print("best rmse val:", math.sqrt(my_model.history.history['val_mean_squared_error'][-1]))